Import libraries:

In [1]:
import re
import nltk
import spacy
import random

import string
from textblob import TextBlob
from nltk.corpus import stopwords
import requests

from allennlp.predictors.predictor import Predictor
import allennlp_models.coref

import pandas as pd
import seaborn as sns
from random import randint
from wordcloud import WordCloud
from tika import parser

from docx import Document
from docx.shared import Cm

from reportlab.lib.styles import ParagraphStyle
from reportlab.platypus import Image
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.enums import TA_JUSTIFY, TA_CENTER
from reportlab.lib.units import inch
from reportlab.lib.pagesizes import letter
from reportlab.platypus import Table, TableStyle

from gensim.summarization import mz_keywords
from gensim.summarization.summarizer import summarize

from selenium import webdriver
from bs4 import BeautifulSoup
import time
from bs4.element import Tag

from pathlib import Path
import wikipediaapi
import concurrent.futures

import winsound
import numpy as np
import pandas as pd
from tqdm import tqdm
import networkx as nx
from requests import get
from googlesearch import search

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize 
import matplotlib.pyplot as plt
from bs4.element import Comment
from bs4 import BeautifulSoup
import urllib.request

import time
import collections
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from nltk import tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from spacy.symbols import nsubj, nsubjpass, VERB, AUX 

from selenium.webdriver.common.keys import Keys
import chromedriver_binary

Import CDFs:

In [2]:
def wiki_scrape(topic_name, verbose=True):
    def wiki_link(link):
        try:
            page = wiki_api.page(link)
            if page.exists():
                d = {'page': link, 'text': page.text, 'link': page.fullurl,
                     'categories': list(page.categories.keys())}
                return d
        except:
            return None

    wiki_api = wikipediaapi.Wikipedia(language='en',
        extract_format=wikipediaapi.ExtractFormat.WIKI)
    page_name = wiki_api.page(topic_name)
    if not page_name.exists():
        return
    page_links = list(page_name.links.keys())
    progress = tqdm(desc='Links Scraped', unit='', total=len(page_links)) if verbose else None
    sources = [{'page': topic_name, 'text': page_name.text, 'link': page_name.fullurl,
                'categories': list(page_name.categories.keys())}]
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_link = {executor.submit(wiki_link, link): link for link in page_links}
        for future in concurrent.futures.as_completed(future_link):
            data = future.result()
            progress.update(1) if verbose else None
            if data:
                sources.append(data)
    progress.close() if verbose else None
    blacklist = ('Template', 'Help:', 'Category:', 'Portal:', 'Wikipedia:', 'Talk:')
    sources = pd.DataFrame(sources)
    sources = sources[(len(sources['text']) > 20)
                      & ~(sources['page'].str.startswith(blacklist))]
    sources['categories'] = sources.categories.apply(lambda x: [y[9:] for y in x])
    sources['topic'] = topic_name
    return sources

def wiki_page(page_name):
    wiki_api = wikipediaapi.Wikipedia(language='en', extract_format=wikipediaapi.ExtractFormat.WIKI)
    page_name = wiki_api.page(page_name)
    if not page_name.exists():
        print('page does not exist')
        return
    page_data = {'page': page_name, 'text': page_name.text, 'link': page_name.fullurl,
                 'categories': [[y[9:] for y in list(page_name.categories.keys())]]}
    page_data_df = pd.DataFrame(page_data)
    return page_data_df

def tags(x):
    return nltk.pos_tag(nltk.word_tokenize(x))[0][1]

def scoring(a,b):
    try:
        a = a.replace(',', '').split()
        b = b.replace(',', '').split()
        score = int(100*len(set(a).intersection(b))/(len(set(a))))
    
    except:
        score = 0    
        
    return score

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_normalize(txt, nouns=True):
    processed_text = re.sub('[^a-zA-Z]', ' ', txt)
    processed_text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",processed_text)
    processed_text=re.sub("(\\d|\\W)+"," ",processed_text)

    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(processed_text) 
    tokens = [lemmatizer.lemmatize(word) for word in tokens if not word in stop_words]
    
    if nouns==True: 
        tokens = [i for i in tokens if ((tags(i) in ['NN', 'NNP', 'NNS', 'NNPS']) and len(i) > 3)]
    
    return tokens

def text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)


def word_cloud(tokens):
    stop_words = set(stopwords.words("english"))
    bigrams_list = list(nltk.bigrams(tokens))
    dictionary = [' '.join(tup) for tup in bigrams_list]

    vectorizer = CountVectorizer(ngram_range=(2, 2))
    bag_of_words = vectorizer.fit_transform(dictionary)
    words_freq = [(word, bag_of_words.sum(axis=0)[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)

    wordcloud = WordCloud(stopwords=stop_words, width=300, height=300, random_state=42, max_words=50, prefer_horizontal=1, 
                          background_color='white', max_font_size=256)
    wordcloud.generate_from_frequencies(dict(words_freq))  
    keys = wordcloud.words_
    
    plt.figure(figsize=(7,7))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.savefig('reports/wordcloud.png')
    plt.show()
    
    return keys 
    
def create_graph(text, common=200):
    tokens = text_normalize(text)
    
    bigrams=list(nltk.ngrams(tokens, 2))
    bigram_counts = collections.Counter(bigrams)
    bigram_df = pd.DataFrame(bigram_counts.most_common(common), columns=['bigram', 'count'])
    d = bigram_df.set_index('bigram').T.to_dict('records')

    F = nx.Graph()
    for k, v in d[0].items():
            F.add_edge(k[0], k[1], weight=(v*10))

            pos = nx.spring_layout(F, iterations=500)
    return F, pos

def parse_page(url):
    htmlString = get(url).text
    soup = BeautifulSoup(htmlString, 'html.parser')
    paragraphs = soup.find_all(['a'])
    txt = text_from_html(str(paragraphs))

    return txt

def get_unique_text(document):
    unique_sentences = []
    for sentence in [sent.raw for sent in TextBlob(document).sentences]:
        if sentence not in unique_sentences:
            unique_sentences.append(sentence)
    return ' '.join(unique_sentences)

def syntax_full(spacy_sentence):
    result=[] 
    for token in spacy_sentence:
        if (token.dep == nsubj or token.dep == nsubjpass) and (token.head.pos == VERB or token.head.pos == AUX):
            result.append(token.head)
    if result:
        return True
    else:
        return False
    
def tree_graph(d):
    valency_list=[d.get(element) for element in d] #list of valency values
    unique_v_list=list(set(valency_list)) #unique valency values

    unique_v_list.reverse()

    N=5 #number of rows
    T=5 #top words

    v_step=[1/N*step for step in range(0,N)] #list of borders for v values
    v_step.reverse()

    d={k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)} #sort d
    levels=[]
    used=[]
    for step in v_step:
        level=[]
        for k in d:
            if d[k]/max(unique_v_list)>step and k not in used:
                level.append(k)
                used.append(k)
        levels.append(level)
    levels=[level for level in levels if level]

    top_levels=[]
    for level in levels:
        if len(level)>T:
            top_levels.append(level[:T])
        else:
            top_levels.append(level)

    levels=top_levels

    gaps=[len(level)-1 for level in levels] #count number of gaps between two words on each level

    lines=[] #get list of strings of every level
    for level in levels:
        level_line=''
        for w in level:
            level_line=level_line+w
        lines.append(level_line)

    longest_symbol_row=max([len(line) for line in lines])+max(gaps)+2 
    ax_step=1/longest_symbol_row #X pos step in the graph

    s=5
    fig = plt.figure(facecolor='#EBF5FB')
    fig.set_size_inches((s+3), s) #set size in inches

    ay_step=1/(len(levels)) #Y pos step in the graph
    i=0
    h_lines_y=[]
    colors=['#D6EAF8','#AED6F1','#85C1E9','#5DADE2','#3498DB']
    
    for level in levels:
        level_line='' #get length of level
        for w in level:
            level_line=level_line+w
        l_step=(longest_symbol_row-len(level_line))/2-gaps[i]/2  #step from the left to start placing words

        h_lines_y.append(1-(i*ay_step))
        color=colors.pop()
        for w in level:
            plt.text(l_step*ax_step+len(w)/2*ax_step, 1-ay_step/2-(i*ay_step), w, fontdict=dict(fontsize=12), 
                     family='monospace', horizontalalignment='center', verticalalignment='center', 
                     bbox=dict(facecolor=color, edgecolor='black', boxstyle='round,pad=0.2'))
            l_step=l_step+len(w)+1
        i=i+1

    h_lines_y.append(0)
    for y in h_lines_y:
      if y==max(h_lines_y) or y==min(h_lines_y):
        plt.axhline(y=y, color='black')
      else:
        plt.axhline(y=y, color='black', ls='dashed')
   
    plt.axvline(x=0,ymin=min(h_lines_y), ymax=max(h_lines_y), color='black')
    plt.axvline(x=1,ymin=min(h_lines_y), ymax=max(h_lines_y), color='black')
    
    plt.axis('off')
    plt.savefig('reports/graph.png')
    plt.show()

Parameters:

In [3]:
en_stopwords = stopwords.words('english') #load stopwords
predictor = Predictor.from_path('./coref-spanbert-large-2020.02.27.tar.gz') #predictor for clustering
top_number = 20 #keywords
number = 20 #pages

error loading _jsonnet (this is expected on Windows), treating C:\Users\SKAMEN~1\AppData\Local\Temp\tmpuvz68amb\config.json as plain json
Did not use initialization regex that was passed: _context_layer._module.weight_hh.*
Did not use initialization regex that was passed: _context_layer._module.weight_ih.*


Create query:

In [111]:
query = input()

wikitext = ''
googletext = ''
arxivtext = ''

urls = []
url_list = []
df = pd.DataFrame()

text summarization


Parse Wikipedia:

In [112]:
try:
    df = wiki_scrape(query, verbose=True)[:number]
    df = df[['text','link']]
    wikitext = '; '.join(list(df['text'])) 
except:
    wikitext = 'no text found'
    print('Text not found')
    
print('Number of Wiki documents:', len(df)) 
winsound.Beep(2500, 1000)

Links Scraped: 100%|███████████████████████████████████████████████████████████████████| 130/130 [00:20<00:00,  6.24/s]


Number of Wiki documents: 20


Parse Google:

In [113]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chrome_options=chrome_options)

google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number+1)
driver.get(google_url)
time.sleep(randint(1,5))

soup = BeautifulSoup(driver.page_source,'lxml')
result_div = soup.find_all('div', attrs={'class': 'g'})

links = []
titles = []
errors = []

descriptions = []
for r in result_div:
    try:
        link = r.find('a', href=True)
        title = None
        title = r.find('h3')

        if isinstance(title,Tag):
            title = title.get_text()

        description = None
        description = r.find('span', attrs={'class': 'st'})

        if isinstance(description, Tag):
            description = description.get_text()

        if link != '' and title != '' and description != '':
            links.append(link['href'])
            titles.append(title)
            descriptions.append(description)

    except Exception as e:
        print(e)
        continue

url_list = links[:(number)]

txt = []
for j in tqdm(url_list):
    delta = random.randint(1,1)
    time.sleep(delta)
    try:  
        if str(j).endswith('.pdf'): 
            file_data = parser.from_file(str(j))           
            t = file_data['content']
        else:
        t = parse_page(j)
        txt.append(t)
    except:
        print('Parsing error:',str(j))
        errors.append(str(j))
       
summ = '; '.join(txt)
googletext = re.sub('[^A-Za-z0-9.]+', ' ', summ)

print('Number of web pages:', len(url_list))
print('Errors:', errors)
winsound.Beep(2500, 1000)

C:\Users\skamenshchikov\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.
 90%|█████████████████████████████████████████████████████████████████████████▊        | 18/20 [00:37<00:04,  2.15s/it]2020-08-29 18:11:55,737 [MainThread  ] [INFO ]  Retrieving https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-864-advanced-natural-language-processing-fall-2005/lecture-notes/lec25.pdf to C:\Users\SKAMEN~1\AppData\Local\Temp/courses-electrical-engineering-and-computer-science-6-864-advanced-natural-language-processing-fall-2005-lecture-notes-lec25.pdf.
INFO:tika.tika:Retrieving https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-864-advanced-natural-language-processing-fall-2005/lecture-notes/lec25.pdf to C:\Users\SKAMEN~1\AppData\Local\Temp/courses-electrical-engineering-and-computer-science-6-864-advanced-natural-language-processing-fall-2005-lecture-

Number of web pages: 20
Errors: []


Parse Arxiv:

In [7]:
closest_value = 200
req = 'https://arxiv.org/search/?query='+query+'&size='+str(closest_value)+'&searchtype=all&source=header&start=0'
htmlString = get(req)

try:
    soup = BeautifulSoup(htmlString.content, 'html5lib') 
    hrefs = soup.find_all('a', {'href': re.compile(r'arxiv.org/abs/')})
    
    if (len(hrefs) > 0):
        for i in hrefs:
            urls.append(i['href'])    

except:
    print ('Connection error')
    
txt = []
for i in tqdm(urls[:number]):
    time.sleep(random.randint(1,8))
    soup = BeautifulSoup(get(str(i)).content, 'html5lib')
    abstract = soup.find('blockquote').text.replace('\n','').replace('  ',' ')
    abstract = ' '.join(abstract.split())
    txt.append(abstract)
         
summ = '; '.join(txt)
arxivtext = re.sub('[^A-Za-z0-9.]+', ' ', summ)
print('Number of web pages:', len(urls[:number]))
winsound.Beep(2500, 1000)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:45<00:00,  5.27s/it]


Number of web pages: 20


Statistics:

In [114]:
docs_length = len(df) + len(url_list) + len(urls)
print('Number of documents:',docs_length)

Number of documents: 40


Aggregate text:

In [115]:
final_text = (wikitext + ' ' + googletext + ' ' + arxivtext)

Filter outliers:

In [116]:
sent_length = [len(i.split()) for i in nltk.sent_tokenize(final_text)] 
lb = q1 -(1.5 * (np.percentile(sent_length, [75]) - np.percentile(sent_length, [25]))) 
ub = q3 +(1.5 * (np.percentile(sent_length, [75]) - np.percentile(sent_length, [25]))) 

filtered_text = [i for i in nltk.sent_tokenize(final_text) if len(i.split()) < lb or len(i.split()) > ub] 
final_text = ' '.join(filtered_text).replace('\n','')

Replacement of reference links:

In [31]:
%%time
span_dict=predictor.predict(document=final_text)
words=span_dict['document']
original_document=span_dict['document']
clusters=span_dict['clusters']

for cluster in clusters:
    first_element=cluster[0]
    head=original_document[first_element[0]:first_element[1]+1] #get head of cluster
    if len(head)==1 and head[0].lower() in en_stopwords: #check if pronoun
        pass
    else:
        for element in cluster:
            for index in range(element[0],element[1]+1):
                words[index]='' #make all elements empty to save indexes of document words
        for element in cluster:
            words[element[0]]="".join([" "+w if not w.startswith("'") and w not in string.punctuation else w for w in head]).strip() #add head as a first word of element

final_text="".join([" "+w if not w.startswith("'") and w!='' and w not in string.punctuation else w for w in words]).strip()

Show text size:

In [117]:
print('Text size:', int(len(final_text.encode('utf-8'))/1024),'mb')

Text size: 119 mb


Syntax filter:

In [118]:
final_text = get_unique_text(final_text)
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 50000000

doc = nlp(final_text)
valid_sents=[]

for s in tqdm(list(doc.sents)):
    if syntax_full(s):
        valid_sents.append(s.text)

final_text = ' '.join(valid_sents)
winsound.Beep(2500, 1000)

100%|███████████████████████████████████████████████████████████████████████████| 1476/1476 [00:00<00:00, 30202.97it/s]


Graph based keywords:

In [119]:
F,pos = create_graph(final_text.lower())

nodes = []
degree = []

for i in F.nodes():   
    nodes.append(i)
    degree.append(F.degree(i))

key_bigrams = []
birams_score = []
bigram_vocab = []

x = dict(zip(nodes, degree))
major_nodes = list({k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)})

for i in major_nodes:
    if i!=list(F.neighbors(i))[0] and (i not in bigram_vocab and list(F.neighbors(i))[0] not in bigram_vocab):
        key_bigrams.append(str(i+' '+list(F.neighbors(i))[0]))
        birams_score.append(F.degree[i])
        bigram_vocab = set((' '.join(key_bigrams)).split())
        
key_bigrams = key_bigrams[:top_number]
birams_score = birams_score[:top_number]

bigram_dict = dict(zip(key_bigrams, birams_score))
normed_keys = []

for i in range(len(bigram_dict.items())):
    normed_keys.append(''.join(str(list(bigram_dict.keys())[i])+' ('+str(list(bigram_dict.values())[i])+'%)'))

key_bigrams = ', '.join(normed_keys)
print('Keywords:\n')
print(key_bigrams)

winsound.Beep(2500, 1000)

Keywords:

displaystyle number (19%), cluster distance (13%), data science (12%), article deep (8%), sentence compression (8%), summarization text (5%), word model (4%), index bouldin (4%), algorithm produce (4%), centroid triangle (4%), june reply (3%), language processing (3%), cream child (3%), question test (3%), matrix entry (3%), recognition speech (3%), object belongs (3%), measure evaluation (3%), machine translation (3%), analysis component (3%)


Get abstract:

In [120]:
readtime = 1
sent_num = int(readtime/(np.median([len(i.split()) for i in nltk.sent_tokenize(final_text)])*0.005))

ratio = sent_num/len(nltk.sent_tokenize(final_text))
txt = summarize(wikitext, ratio=ratio)

print('Compression:', round(100*ratio,1), '%\n')
[print('* ' + i) for i in nltk.sent_tokenize(txt)]

winsound.Beep(2500, 1000)

Compression: 3.7 %

* Automatic summarization is the process of shortening a set of data computationally, to create a subset (a summary) that represents the most important or relevant information within the original content.
* Examples of extracted content include key-phrases that can be used to "tag" or index a text document, or key sentences (including headings) that collectively comprise an abstract, and representative images or video segments, as stated above.
* For text, extraction is analogous to the process of skimming, where the summary (if available), headings and subheadings, figures, the first and last paragraphs of a section, and optionally the first and last sentences in a paragraph are read before one chooses to read the entire document in detail.
* Abstractive methods build an internal semantic representation of the original content, and then use this representation to create a summary that is closer to what a human might express.
* Such transformation, however, is compu

Create PDF report:

In [121]:
pdf_name = 'reports/summary.pdf'

short_summary = '<br/><br/>'.join([('* ' + i) for i in nltk.sent_tokenize(txt)])

sp = ParagraphStyle('parrafos', alignment=TA_JUSTIFY, fontSize=12, fontName="Times-Roman")
headers = ParagraphStyle('parrafos', alignment=TA_CENTER, fontSize=14, fontName="Times-Roman")
doc = SimpleDocTemplate(pdf_name, pagesize=letter, bottomMargin=.4 * inch, topMargin=.6 * inch, rightMargin=.8 * inch, leftMargin=.8 * inch)

hd = Paragraph('<b>Summary</b>', headers)
P = Paragraph(short_summary, sp)
kw = Paragraph(('\n<b>Keywords</b>: ' + str(key_bigrams) + '.'), sp)
rq = Paragraph(('\n<br /><b>Request</b>: ' + str(query)), sp)
pn = Paragraph(('\n<br /><b>Number of documents:</b>: ' + str(docs_length)), sp)
ts = Paragraph(('\n<br /><b>Text size:</b> ' + str(int(len(final_text.encode('utf-8'))/1024))) + ' mb', sp)
rq = Paragraph(('\n<br /><b>Number of pages</b>: ' + str(query)), sp)
cmp = Paragraph(('\n<br /><b>Number of sentences</b>: ' + str(sent_num)), sp)

rs = Paragraph(('\n<br /><b>Resources</b>: ' + str('Wiki, Arxiv, Google')), sp)
space = Paragraph('<br /><br />', sp)

catalog = []
img_hist = Image('reports/hist.png', 3.25*inch, 6.3*inch)

catalog.append(hd)
catalog.append(rq)
catalog.append(rs)
catalog.append(pn)
catalog.append(cmp)
catalog.append(ts)
catalog.append(space)
catalog.append(kw)
catalog.append(space)
catalog.append(P)

doc.build(catalog)

Create DOCX report:

In [122]:
short_summary = '\n'.join([('* ' + i + '\n') for i in nltk.sent_tokenize(txt)])

doc = Document()
hd = doc.add_paragraph()
hd.add_run('Summary').bold = True
hd = doc.add_paragraph(re.sub(r'[^\x00-\x7F]+|\x0c',' ', ('Request: ' + str(query))))
hd = doc.add_paragraph(re.sub(r'[^\x00-\x7F]+|\x0c',' ', ('Resources: ' + str('Wiki, ' + 'Arxiv, ' + 'Google'))))
hd = doc.add_paragraph(re.sub(r'[^\x00-\x7F]+|\x0c',' ', ('Number of documents: ' + str(docs_length))))
hd = doc.add_paragraph(re.sub(r'[^\x00-\x7F]+|\x0c',' ', ('Text size: ' + str(int(len(final_text.encode('utf-8'))/1024)) + ' mb')))
hd = doc.add_paragraph(re.sub(r'[^\x00-\x7F]+|\x0c',' ', ('Keywords: ' + str(key_bigrams))))

hd.alignment = 3

r = hd.add_run()
par = doc.add_paragraph(re.sub(r'[^\x00-\x7F]+|\x0c',' ', (short_summary.replace('.','. '))))
par.alignment = 0

doc.save('reports/summary.docx')